In [1]:
import tensorflow as tf
import timeit
from datetime import datetime

# 그래프 추적 하기

In [2]:
def function_to_get_faster(x, y, b):
    return tf.matmul(x,y) + b

a_function_that_uses_a_graph = tf.function(function_to_get_faster)

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

a_function_that_uses_a_graph(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

In [3]:
def inner_function(x, y, b):
    return tf.matmul(x,y) + b

@tf.function
def outer_function(x):
    y = tf.constant([[2.0], [3.0]])
    b = tf.constant(4.0)
    return inner_function(x, y, b)
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

# 흐름 제어 및 부작용

In [4]:
def my_function(x):
    if tf.reduce_sum(x) <= 1:
        return x * x
    else:
        return x - 1

a_function = tf.function(my_function)

print("First branch, with graph:", a_function(tf.constant(1.0)).numpy())
print("Second branch, with graph:", a_function(tf.constant([5.0, 5.0])).numpy())

First branch, with graph: 1.0
Second branch, with graph: [4. 4.]


In [5]:
# Don't read the output too carefully.
print(tf.autograph.to_code(my_function))

def tf__my_function(x):
    with ag__.FunctionScope('my_function', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = (ag__.ld(x) * ag__.ld(x))
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = (ag__.ld(x) - 1)
            except:
                do_return = False
                raise
        ag__.if_stmt((ag__.converted_call(ag__.ld(tf).reduce_sum, (ag

# 속도 향상

In [6]:
class SequentialModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(SequentialModel, self).__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten(input_shape = (28, 28))
        self.dense_1 = tf.keras.layers.Dense(128, activation = "relu")
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.dense_2 = tf.keras.layers.Dense(10)

    def call(self, x):
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout(x)
        x = self.dense_2(x)
        return x

input_data = tf.random.uniform([60, 28, 28])

eager_model = SequentialModel()
graph_model = tf.function(eager_model)

print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=10000))
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=10000))

Eager time: 11.185795360999919
Graph time: 5.279192726000019


# 다형 함수

In [18]:
print(a_function)
print("Calling a `Function`:")
print("Int:", a_function(tf.constant(2)))
print("Float:", a_function(tf.constant(2.0)))
print("Rank-1 tensor of floats", a_function(tf.constant([2.0, 2.0, 2.0])))

Calling a `Function`:
Int: tf.Tensor(1, shape=(), dtype=int32)
Float: tf.Tensor(1.0, shape=(), dtype=float32)
Rank-1 tensor of floats tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)


In [22]:
# Get the concrete function that works on floats
# print("Inspecting concrete functions")
# print("Concrete function for float:")
# print()
print(a_function.get_concrete_function(tf.TensorSpec(shape=[], dtype=tf.float32)))
print()
# print("Concrete function for tensor of floats:")
print()
print(a_function.get_concrete_function(tf.constant([2.0, 2.0, 2.0])))

ConcreteFunction my_function(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()


ConcreteFunction my_function(x)
  Args:
    x: float32 Tensor, shape=(3,)
  Returns:
    float32 Tensor, shape=(3,)


In [23]:
# Concrete functions are callable
# Note: You won't normally do this, but instead just call the containing `Function`
cf = a_function.get_concrete_function(tf.constant(2))
print("Directly calling a concrete function:", cf(tf.constant(2)))

Directly calling a concrete function: tf.Tensor(1, shape=(), dtype=int32)


# 즉시 실행으로 되돌리기

In [24]:
class EagerLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(EagerLayer, self).__init__(**kwargs)
    def call(self, inputs):
        print("\nCurrently running eagerly", str(datetime.now()))
        return inputs

In [28]:
# 모델 구조 생성
class SequentialModel(tf.keras.Model):
    def __init__(self):
        super(SequentialModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.dense_2 = tf.keras.layers.Dense(10)
        self.eager = EagerLayer()
    def call(self, x):
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout(x)
        x = self.dense_2(x)
        return self.eager(x)

# 인스턴스화
model = SequentialModel()

# 데이터와 손실함수 생성
input_data = tf.random.uniform([60, 28, 28])
labels = tf.random.uniform([60])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
# 모델에 대한 설정
model.compile(run_eagerly=False, loss=loss_fn)

In [34]:
model.fit(input_data,labels,epochs=3)

Epoch 1/3

Currently running eagerly 2021-06-01 05:21:22.320395

Currently running eagerly 2021-06-01 05:21:22.444270
2/2 [==============================] - 0s 3ms/step - loss: 6.4541e-04
Epoch 2/3
2/2 [==============================] - 0s 3ms/step - loss: 1.9570e-06
Epoch 3/3
2/2 [==============================] - 0s 3ms/step - loss: 9.9540e-07


In [35]:
# Now, globally set everything to run eagerly
tf.config.experimental_run_functions_eagerly(True)
print("Run all functions eagerly.")

# First, trace the model, triggering the side effect
polymorphic_function = tf.function(model)

# It was traced...
print(polymorphic_function.get_concrete_function(input_data))

# But when you run the function again, the side effect happens (both times).
result = polymorphic_function(input_data)
result = polymorphic_function(input_data)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
Run all functions eagerly.

Currently running eagerly 2021-06-01 05:28:41.317113
ConcreteFunction function(<arg1>)
  Args:
    <arg1>: float32 Tensor, shape=(60, 28, 28)
  Returns:
    float32 Tensor, shape=(60, 10)

Currently running eagerly 2021-06-01 05:28:41.323751

Currently running eagerly 2021-06-01 05:28:41.325752


# 추적 및 성능

In [38]:
# Use @tf.function decorator
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!")  # This eager
  return x * x + tf.constant(2)

# This is traced the first time
print(a_function_with_python_side_effect(tf.constant(2)))
# The second time through, you won't see the side effect
print(a_function_with_python_side_effect(tf.constant(3)))

# This retraces each time the Python argument chances
# as a Python argument could be an epoch count or other
# hyperparameter
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))

Tracing!
tf.Tensor(6, shape=(), dtype=int32)
Tracing!
tf.Tensor(11, shape=(), dtype=int32)
Tracing!
tf.Tensor(6, shape=(), dtype=int32)
Tracing!
tf.Tensor(11, shape=(), dtype=int32)
